In [66]:
import logging
logging.basicConfig()

from kazoo.client import KazooClient
zk = KazooClient()
zk.start()

In [67]:
zk.get_children("/")

['zookeeper', 'task1', 'task_2', 'myconfig']

In [68]:
zk.get_children("/zookeeper"), zk.get_children("/zookeeper/quota")

(['quota'], [])

In [69]:
zk.delete(path="/mynode", recursive= True)
zk.create(path="/mynode", value=b"first_version")

'/mynode'

In [70]:
zk.get_children("/")

['mynode', 'zookeeper', 'task1', 'task_2', 'myconfig']

In [71]:
zk.get('/mynode') 

(b'first_version',
 ZnodeStat(czxid=4140, mzxid=4140, ctime=1669789161726, mtime=1669789161726, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=4140))

In [72]:
zk.set('/mynode', value=b"second_version")

ZnodeStat(czxid=4140, mzxid=4141, ctime=1669789161726, mtime=1669789162603, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=14, numChildren=0, pzxid=4140)

In [73]:
zk.get('/mynode') 

(b'second_version',
 ZnodeStat(czxid=4140, mzxid=4141, ctime=1669789161726, mtime=1669789162603, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=14, numChildren=0, pzxid=4140))

In [74]:
zk.create('/mynode/child', value=b'im_sequential', sequence=True)
zk.create('/mynode/child', value=b'me_too', sequence=True)

'/mynode/child0000000001'

In [75]:
zk.get_children('/mynode')

['child0000000000', 'child0000000001']

In [28]:
zk.delete('/mynode', recursive= True)

# Эфимерные узлы

In [17]:
zk1, zk2 = KazooClient(), KazooClient()
zk1.start()
zk2.start()

In [19]:
zk.delete('/mygroup', recursive= True)
zk.create('/mygroup', b'top_node')

'/mygroup'

In [20]:
zk1.create('/mygroup/grue', b'im_grue', ephemeral=True)
zk2.create('/mygroup/bleen', b'im_bleen', ephemeral=True)

'/mygroup/bleen'

In [21]:
zk.get_children('/mygroup')

['grue', 'bleen']

In [22]:
 zk1.get('/mygroup/bleen')

(b'im_bleen',
 ZnodeStat(czxid=4110, mzxid=4110, ctime=1669788542374, mtime=1669788542374, version=0, cversion=0, aversion=0, ephemeralOwner=72059978472882182, dataLength=8, numChildren=0, pzxid=4110))

In [23]:
zk2.stop()

In [24]:
zk.get_children('/mygroup')

['grue']

In [25]:
zk1.get('/mygroup/bleen')

NoNodeError: 

In [26]:
zk1.stop(), zk2.stop()

(None, None)

In [27]:
 zk.delete('/mygroup', recursive=True)

# Конфигурация распределённого приложения

In [49]:
zk.delete('/myconfig', recursive=True)
zk.create('/myconfig', b'sheep_count=1')

'/myconfig'

In [50]:
zk1 = KazooClient()
zk1.start()

In [51]:
def watch_node(state):
    print(state)

In [52]:
zk1.get('/myconfig', watch=watch_node)

(b'sheep_count=1',
 ZnodeStat(czxid=4135, mzxid=4135, ctime=1669788703449, mtime=1669788703449, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=4135))

In [53]:
zk.set('/myconfig', b'sheep_count=2')

WatchedEvent(type='CHANGED', state='CONNECTED', path='/myconfig')


ZnodeStat(czxid=4135, mzxid=4137, ctime=1669788703449, mtime=1669788704239, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=4135)

In [54]:
zk.set('/myconfig', b'sheep_count=3')  # Trigger (watcher) is resetted 

ZnodeStat(czxid=4135, mzxid=4138, ctime=1669788703449, mtime=1669788704701, version=2, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=4135)

# Мониторим ZooKeeper

In [58]:
!echo ruok | nc localhost 2181 

imok

In [59]:
!echo stat | nc localhost 2181

Zookeeper version: 3.4.14-4c25d480e66aadd371de8bd2fd8da255ac140bcf, built on 03/06/2019 16:18 GMT
Clients:
 /127.0.0.1:42818[0](queued=0,recved=1,sent=0)
 /127.0.0.1:42806[1](queued=0,recved=46,sent=48)
 /127.0.0.1:42786[1](queued=0,recved=193,sent=193)
 /127.0.0.1:42804[1](queued=0,recved=60,sent=61)
 /127.0.0.1:42664[1](queued=0,recved=176,sent=182)
 /127.0.0.1:42808[1](queued=0,recved=36,sent=37)

Latency min/avg/max: 0/0/33
Received: 595
Sent: 610
Connections: 6
Outstanding: 0
Zxid: 0x102a
Mode: standalone
Node count: 15


In [61]:
!echo conf | nc localhost 2181

clientPort=2181
dataDir=/data/version-2
dataLogDir=/datalog/version-2
tickTime=2000
maxClientCnxns=60
minSessionTimeout=4000
maxSessionTimeout=40000
serverId=0


In [62]:
!echo cons | nc localhost 2181

 /127.0.0.1:42822[0](queued=0,recved=1,sent=0)
 /127.0.0.1:42806[1](queued=0,recved=64,sent=66,sid=0x100022b2b660008,lop=PING,est=1669788668624,to=10000,lcxid=0x2,lzxid=0x102a,lresp=37112518,llat=0,minlat=0,avglat=0,maxlat=2)
 /127.0.0.1:42786[1](queued=0,recved=210,sent=210,sid=0x100022b2b660004,lop=PING,est=1669788361937,to=10000,lcxid=0x41,lzxid=0x102a,lresp=37111550,llat=0,minlat=0,avglat=0,maxlat=33)
 /127.0.0.1:42804[1](queued=0,recved=77,sent=78,sid=0x100022b2b660007,lop=PING,est=1669788626180,to=10000,lcxid=0x1,lzxid=0x102a,lresp=37110732,llat=1,minlat=0,avglat=0,maxlat=2)
 /127.0.0.1:42664[1](queued=0,recved=194,sent=200,sid=0x100022b2b660000,lop=PING,est=1669788333328,to=10000,lcxid=0x18,lzxid=0x102a,lresp=37111939,llat=0,minlat=0,avglat=0,maxlat=4)
 /127.0.0.1:42808[1](queued=0,recved=54,sent=55,sid=0x100022b2b660009,lop=PING,est=1669788703647,to=10000,lcxid=0x1,lzxid=0x102a,lresp=37111652,llat=1,minlat=0,avglat=0,maxlat=1)



In [63]:
!echo stat | nc localhost 2181

Zookeeper version: 3.4.14-4c25d480e66aadd371de8bd2fd8da255ac140bcf, built on 03/06/2019 16:18 GMT
Clients:
 /127.0.0.1:42824[0](queued=0,recved=1,sent=0)
 /127.0.0.1:42806[1](queued=0,recved=71,sent=73)
 /127.0.0.1:42786[1](queued=0,recved=217,sent=217)
 /127.0.0.1:42804[1](queued=0,recved=84,sent=85)
 /127.0.0.1:42664[1](queued=0,recved=201,sent=207)
 /127.0.0.1:42808[1](queued=0,recved=61,sent=62)

Latency min/avg/max: 0/0/33
Received: 721
Sent: 736
Connections: 6
Outstanding: 0
Zxid: 0x102a
Mode: standalone
Node count: 15


In [64]:
!echo dump | nc localhost 2181

SessionTracker dump:
Session Sets (5):
0 expire at Thu Jan 01 10:19:06 UTC 1970:
0 expire at Thu Jan 01 10:19:08 UTC 1970:
0 expire at Thu Jan 01 10:19:10 UTC 1970:
0 expire at Thu Jan 01 10:19:12 UTC 1970:
5 expire at Thu Jan 01 10:19:14 UTC 1970:
	0x100022b2b660004
	0x100022b2b660000
	0x100022b2b660009
	0x100022b2b660007
	0x100022b2b660008
ephemeral nodes dump:
Sessions with Ephemerals (0):


In [65]:
!echo wchs | nc localhost 2181

3 connections watching 0 paths
Total watches:0
